In [ ]:
import numpy as np
import pygetm

%matplotlib widget
import matplotlib.pyplot

## Set up the domain (Cartesian box)

In [ ]:
Lx, Ly = 100.0, 100.0
nx, ny = 100, 101
x = np.linspace(-Lx / 2, Lx / 2, nx)
y = np.linspace(-Ly / 2, Ly / 2, ny)
domain = pygetm.domain.create_cartesian(x, y, nz=1, H=1.0, lat=0.0)
domain.initialize(runtype=pygetm.BAROTROPIC_2D)

## Configure rotational velocity field

In [ ]:
# Parameters
period = 600
omega = 2 * np.pi / period
cfl = 1.0
umax = omega * Lx / 2
dt_cfl = cfl * min(Lx / (nx + 1), Ly / (ny + 1)) / umax
no_of_revolutions = 5
Nmax = no_of_revolutions * round(2 * np.pi / omega / dt_cfl)
tmax = no_of_revolutions * 2 * np.pi / omega
timestep = tmax / Nmax

# 2D velocities
u = domain.U.array(fill=0.0)
v = domain.V.array(fill=0.0)
u[:, :] = -omega * domain.U.y
v[:, :] = omega * domain.V.x
u[(2 * domain.U.x / Lx) ** 2 + (2 * domain.U.y / Ly) ** 2 > 1] = 0.0
v[(2 * domain.V.x / Lx) ** 2 + (2 * domain.V.y / Ly) ** 2 > 1] = 0.0

# Plot velocity field
fig, ax = matplotlib.pyplot.subplots()
u_destag, v_destag = u.interp(domain.T), v.interp(domain.T)
pc = ax.quiver(u_destag[::5, ::5], v_destag[::5, ::5])

## Set up square tracer field

In [ ]:
var = domain.T.array(fill=1.0)
var[int(0.2 * ny) : int(0.4 * ny), int(0.2 * nx) : int(0.4 * nx),] = 5.0

# Plot
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(var)
cb = fig.colorbar(pc)
title = ax.set_title("time step: 0 of %i" % Nmax)

## Simulate and animate

In [ ]:
nsample = 4
totals = [var[...].sum()]
    
advect = pygetm.operators.Advection(domain.T, scheme=pygetm.AdvectionScheme.SUPERBEE)
for i in range(1, Nmax):
    advect(u, v, timestep, var)
    if i % nsample == 0:
        pc.set_array(var[...].ravel())
        title.set_text("time step: %i of %i" % (i, Nmax))
        fig.canvas.draw()
    totals.append(var[...].sum())

## Change in total tracer

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
ax.plot(totals / totals[0] - 1.0)
ax.grid()
ax.set_xlabel("time step")
ax.set_ylabel("change in total tracer")

## Horizontal diffusion only

In [ ]:
# Initialize tracer
var[:, :] = 0.0
var[50, 50] = 1

# Zero velocities
u[:,:] = 0
v[:,:] = 0
Ah = domain.T.array(fill_value=0.01)
Ah_u = domain.U.array(fill_value=0.01)
Ah_v = domain.V.array(fill_value=0.01)

# Plot
fig, (ax1, ax2, ax3) = matplotlib.pyplot.subplots(ncols=3, figsize=(12,4))
pc1 = ax1.pcolormesh(var)
cb = fig.colorbar(pc1, ax=ax1)
pc2 = ax2.pcolormesh(var)
cb = fig.colorbar(pc2, ax=ax2)
pc3 = ax3.pcolormesh(np.zeros_like(var), cmap='bwr')
cb = fig.colorbar(pc3, ax=ax3)
title = ax.set_title("time step: 0 of %i" % Nmax)

In [ ]:
nsample = 4
advect = pygetm.operators.Advection(domain.T, scheme=pygetm.AdvectionScheme.UPSTREAM)
for i in range(1, Nmax):
    advect(u, v, timestep, var, Ah_u=Ah_u, Ah_v=Ah_v)
    if i % nsample == 0:
        xa = domain.T.x.values - domain.T.x.values[50, 50]
        ya = domain.T.y.values - domain.T.y.values[50, 50]
        va = 2 * Ah.values * i * timestep
        analytical = 1. / (2 * np.pi * va) * np.exp(-0.5 * (xa**2 + ya**2) / va) * domain.T.area.values
        diff = analytical - var.values
        pc1.set_array(var[...].ravel())
        pc2.set_array(analytical[...].ravel())
        pc3.set_array(diff.ravel())
        pc1.set_clim(0., var.values.max())
        pc2.set_clim(0., analytical.max())
        maxdiff = np.abs(diff).max()
        pc3.set_clim(-maxdiff, maxdiff)
        title.set_text("time step: %i of %i" % (i, Nmax))
        fig.canvas.draw()